In [1]:
import pandas as pd
import duckdb
import json
import warnings

In [61]:
proposals_data_path = 'data/01-cws_proposals_data.json'
with open(proposals_data_path) as data:
    proposals = json.load(data)
    
error_log_path = 'data/02-cws_proposals_error_log.json'
with open(error_log_path) as data:
    errors = json.load(data)
    
user_data = pd.read_csv('secrets/cws_user_data_merged.tsv', sep='\t', index_col=0)

# Processing functions

In [62]:
def process_proposals(data):
    rows = []
    
    for year, categories in data.items():
        for category, proposals in categories.items():
            for proposal in proposals:
                rows.append({'year': year, 'category': category, 'proposal': proposal})
    
    return pd.DataFrame(rows)

def process_participant_roles(data):
    rows = []
    
    roles_mapping = {
        'proposer': 'proposer',
        'discussion_participants': 'discussant',
        'voters': 'voter'
    }
    
    for year, categories in data.items():
        for category, proposals in categories.items():
            for proposal, details in proposals.items():
                for role_key, role_singular in roles_mapping.items():
                    rows.append({
                        'year': year,
                        'wish': proposal,
                        'role': role_singular,
                        'username': details[role_key] if role_key in details else []
                    })
    
    df = pd.DataFrame(rows)

    df = df.explode('username', ignore_index=True)
    
    return df

def process_phab_tickets(data):
    rows = []
    
    for year, categories in data.items():
        for category, proposals in categories.items():
            for proposal, details in proposals.items():
                rows.append({
                    'year': year,
                    'proposal': proposal,
                    'phab_tickets': details.get('phab_tickets', [])
                })
    
    return pd.DataFrame(rows)

def process_user_data(df):
    df['edit_count'] = df.apply(lambda row: {'hw': row['hw_edit_count'], 'mw': row['mw_edit_count']}, axis=1)
    df['edit_bucket'] = df.apply(lambda row: {'hw': row['hw_edit_bucket'], 'mw': row['mw_bucket']}, axis=1)
    df['account_age'] = df.apply(lambda row: {'months': row['account_age_months'], 'years': row['account_age_years']}, axis=1)

    df = df.drop(columns=['hw_edit_count', 'mw_edit_count', 'hw_edit_bucket', 'mw_bucket', 'account_age_months', 'account_age_years'])
    
    return df

def process_rejects(data):
    rows = []
    
    for year, categories in data.items():
        for category, proposals in categories.items():
            for proposal, details in proposals.items():
                try:
                    rows.append({
                        'year': year,
                        'proposal': proposal,
                        'reason': details.get('reject_reason', 0)
                    })
                except:
                    pass
    
    return pd.DataFrame(rows).query("""reason != 0""").reset_index(drop=True)

def process_error_log(data):
    rows = []

    for year, proposals in data.items():
        for proposal_name, details in proposals.items():
            category = details['category']
            error = details['error']
            rows.append({'year': year, 
                         'proposal_name': proposal_name, 
                         'category': category, 
                         'error': error})
                
    return pd.DataFrame(rows)

# Write to DB

In [72]:
conn = duckdb.connect('secrets/cws_data.db')

In [25]:
proposals_by_year = process_proposals(proposals)

conn.execute("""
CREATE OR REPLACE TABLE proposals AS 
SELECT
    year::INTEGER AS year,
    category,
    proposal
FROM
    proposals_by_year
""")

In [11]:
participant_roles = process_participant_roles(proposals)

conn.execute("""
CREATE OR REPLACE TABLE roles AS 
SELECT
    year::INTEGER AS year,
    wish AS proposal,
    role,
    username
FROM
    participant_roles
""")

In [7]:
phab_tickets = process_phab_tickets(proposals)

conn.execute("""
CREATE OR REPLACE TABLE phab_tickets AS 
SELECT
    year::INTEGER AS year,
    proposal,
    phab_tickets
FROM
    phab_tickets
""")

In [17]:
user_data_processed = process_user_data(user_data)

conn.execute("""
CREATE OR REPLACE TABLE user_data AS 
SELECT
    survey_year::INTEGER AS year,
    username,
    home_wiki,
    hw_user_groups,
    edit_count,
    edit_bucket,
    account_age
FROM
    user_data_processed
""")

In [58]:
rejected_proposals = process_rejects(proposals)

conn.execute("""
CREATE OR REPLACE TABLE rejects AS 
SELECT
    year::INTEGER AS year,
    proposal,
    reason
FROM
    rejected_proposals
""")

In [68]:
error_log = process_error_log(errors)

conn.execute("""
CREATE OR REPLACE TABLE errors AS 
SELECT
    year::INTEGER AS year,
    category,
    proposal_name,
    error
FROM
    error_log
""")

# Check data types

In [73]:
conn.sql("""DESCRIBE proposals""").show()
conn.sql("""DESCRIBE roles""").show()
conn.sql("""DESCRIBE phab_tickets""").show()
conn.sql("""DESCRIBE user_data""").show()
conn.sql("""DESCRIBE rejects""").show()
conn.sql("""DESCRIBE errors""").show()

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ year        │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ category    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ proposal    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ year        │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ proposal    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ role        │ VARCHAR     │ YES     │ NULL   

In [74]:
conn.close()